In [196]:
import os
import sys
import pandas as pd
import requests
from igdb.wrapper import IGDBWrapper
import json
import matplotlib.pyplot as plt
sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")

# Function that creates a list of features
def get_list_of_features(the_json):
    feature_dic = {"id":None,
        "aggregated_rating": None,
        "aggregated_rating_count": None,
        "game_engines": None,
        "game_modes": None,
        "player_perspectives": None,
        "themes": None,
        'rating': None}

    for key in the_json.keys():
        if type(the_json[key]) == list:
            for i in range(len(the_json[key])):
                value = the_json[key][i]['name']
                feature_dic.update({f'{key}': value})

        else:
            value = the_json[key]
            feature_dic.update({f'{key}': value})

    return feature_dic


def get_category_descriptions(json_dict):
    category_list = []
    
    if type(json_dict) == float:
        category_list = None
    else:
        for i in range(len(json_dict)):
            try:
                descriptions = json_dict[i]['content_descriptions']
                for i in range(len(descriptions)):
                    rating_descript = descriptions[i]['category']
                    category_list.append(rating_descript)
                    
            except:
                pass
    
    return category_list


In [197]:
json_data = pd.read_json('../raw_data/api_json_0to30k')
json_data

,id,game_modes,name,themes,player_perspectives,game_engines,age_ratings,multiplayer_modes,aggregated_rating,aggregated_rating_count,rating
0,123757,"[{'id': 1, 'name': 'Single player'}]",PilotXross,"[{'id': 1, 'name': 'Action'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,106694,"[{'id': 1, 'name': 'Single player'}]",Plague Hunter,"[{'id': 1, 'name': 'Action'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,259874,NaN,The God Slayer,"[{'id': 1, 'name': 'Action'}, {'id': 38, 'name...","[{'id': 2, 'name': 'Third person'}]",NaN,NaN,NaN,NaN,NaN,NaN
3,211568,"[{'id': 1, 'name': 'Single player'}]",Rise of Rebellion,"[{'id': 1, 'name': 'Action'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,214912,"[{'id': 1, 'name': 'Single player'}]",Penny Blood,"[{'id': 17, 'name': 'Fantasy'}, {'id': 19, 'na...","[{'id': 2, 'name': 'Third person'}]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29451,150009,"[{'id': 5, 'name': 'Massively Multiplayer Onli...",Moonlight Sculptor,"[{'id': 17, 'name': 'Fantasy'}, {'id': 33, 'na...","[{'id': 2, 'name': 'Third person'}]",NaN,"[{'id': 52219, 'content_descriptions': [{'id':...",[15029],NaN,NaN,NaN
29452,112887,"[{'id': 1, 'name': 'Single player'}, {'id': 3,...",Susume!! Mamotte Knight: Hime no Totsugeki Ser...,"[{'id': 17, 'name': 'Fantasy'}]","[{'id': 3, 'name': 'Bird view / Isometric'}]",NaN,"[{'id': 97295, 'content_descriptions': [{'id':...",NaN,NaN,NaN,NaN
29453,188135,"[{'id': 1, 'name': 'Single player'}]",Moon,"[{'id': 17, 'name': 'Fantasy'}, {'id': 27, 'na...","[{'id': 3, 'name': 'Bird view / Isometric'}]","[{'id': 13, 'name': 'Unity'}]","[{'id': 55594, 'rating': 2}, {'id': 98221, 'ra...",NaN,NaN,NaN,NaN
29454,124986,"[{'id': 1, 'name': 'Single player'}]",CrunchTime,"[{'id': 1, 'name': 'Action'}]","[{'id': 4, 'name': 'Side view'}]",NaN,"[{'id': 38527, 'content_descriptions': [{'id':...",NaN,NaN,NaN,NaN


In [198]:
working_json_df = json_data

In [199]:
dict_list = json_data.iloc[20000]['themes']

In [200]:
def get_names_from_dict(dict_list):
    list_of_items = []
    try:
        for i in range(len(dict_list)):
            item = dict_list[i]['name']
            list_of_items.append(item)
    except:
        list_of_items = None
    
    return list_of_items

In [201]:
get_names_from_dict(dict_list)

['Action', 'Fantasy', 'Drama']

In [202]:
json_data['themes'].apply(get_names_from_dict)

0                                      [Action]
1                                      [Action]
2                          [Action, Open world]
3                                      [Action]
4        [Fantasy, Horror, Historical, Mystery]
                          ...                  
29451                        [Fantasy, Sandbox]
29452                                 [Fantasy]
29453                         [Fantasy, Comedy]
29454                                  [Action]
29455                                      None
Name: themes, Length: 29456, dtype: object

In [203]:
def prepare_json_df(original_json_df):
    
    original_json_df['themes'] = original_json_df['themes'].apply(get_names_from_dict)
    original_json_df['game_modes'] = original_json_df['game_modes'].apply(get_names_from_dict)
    original_json_df['player_perspectives'] = original_json_df['player_perspectives'].apply(get_names_from_dict)
    original_json_df['game_engines'] = original_json_df['game_engines'].apply(get_names_from_dict)
    original_json_df['age_ratings'] = original_json_df['age_ratings'].apply(get_category_descriptions)
    
    return original_json_df.drop('multiplayer_modes', axis=1)



In [204]:
prepare_json_df(json_data)

,id,game_modes,name,themes,player_perspectives,game_engines,age_ratings,aggregated_rating,aggregated_rating_count,rating
0,123757,[Single player],PilotXross,[Action],None,None,None,NaN,NaN,NaN
1,106694,[Single player],Plague Hunter,[Action],None,None,None,NaN,NaN,NaN
2,259874,None,The God Slayer,"[Action, Open world]",[Third person],None,None,NaN,NaN,NaN
3,211568,[Single player],Rise of Rebellion,[Action],None,None,None,NaN,NaN,NaN
4,214912,[Single player],Penny Blood,"[Fantasy, Horror, Historical, Mystery]",[Third person],None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
29451,150009,[Massively Multiplayer Online (MMO)],Moonlight Sculptor,"[Fantasy, Sandbox]",[Third person],None,[50],NaN,NaN,NaN
29452,112887,"[Single player, Co-operative]",Susume!! Mamotte Knight: Hime no Totsugeki Ser...,[Fantasy],[Bird view / Isometric],None,[60],NaN,NaN,NaN
29453,188135,[Single player],Moon,"[Fantasy, Comedy]",[Bird view / Isometric],[Unity],"[7, 29, 45, 3]",NaN,NaN,NaN
29454,124986,[Single player],CrunchTime,[Action],[Side view],None,"[9, 65]",NaN,NaN,NaN
